In [ ]:
# Source Code adapted from: Deep Learning with Python, Second Edition by Francois Chollet

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)
train_data.shape

2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


(8982,)

In [ ]:
#Multi-hot encoding on the training data
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. # the dot is to make sure the datatype is floating point
    return results

X_train = vectorize_sequences(train_data)
X_test = vectorize_sequences(test_data)
X_train.shape

(8982, 10000)

In [ ]:
# Optional: One-hot encoding the labels
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

y_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
import keras

def build_model():
  return keras.Sequential([
      keras.layers.Dense(64, activation="relu"),
      keras.layers.Dense(64, activation="relu"),
      keras.layers.Dense(46, activation="softmax")
  ])
model = build_model()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=30, batch_size=512, validation_split=0.2)

Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.2998 - loss: 3.6380 - val_accuracy: 0.5620 - val_loss: 2.7980
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.5775 - loss: 2.4585 - val_accuracy: 0.6516 - val_loss: 1.7869
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.6806 - loss: 1.5611 - val_accuracy: 0.7045 - val_loss: 1.4124
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.7451 - loss: 1.1896 - val_accuracy: 0.7368 - val_loss: 1.2342
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.7979 - loss: 0.9552 - val_accuracy: 0.7624 - val_loss: 1.1383
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.8395 - loss: 0.7856 - val_accuracy: 0.7746 - val_loss: 1.0862
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.8699 - loss: 0.6282 - val_accuracy: 0.7858 - val_loss: 1.0301
Epoch 8/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.8991 - loss: 0.5281 - val_accuracy: 0.7924 - v

In [ ]:
# evaluate on the test set
results = model.evaluate(X_test, y_test)
results

71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7952 - loss: 1.1403


[1.1465257406234741, 0.7898486256599426]

In [ ]:
# Saving a model
model.save("my_model.keras")

In [ ]:
# load a model
model2 = keras.models.load_model("my_model.keras")

# try it out on the test set without training
results = model2.evaluate(X_test, y_test)
results

71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7914 - loss: 1.1136


[1.1442506313323975, 0.7845057845115662]

# Callbacks
Callbacks are objects that are called by the model at various points during training. They can monitor the model training process to do early stopping, checkpoints, and more.

In [ ]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy", # the metric that we are monitoring
        patience=3, # if the val accuracy fails to improve 3 epochs
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_model.keras",
        monitor="val_loss",
        save_best_only=True, # replace the current save only when the val loss is better
    ),
    keras.callbacks.CSVLogger(
        "training_log.csv", # Output CSV file
        separator=",", # Delimiter
        append=False # Overwrite the file at the start of training
    )
]
model3 = build_model()
model3.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model3.fit(X_train, y_train, epochs=30, batch_size=512, validation_split=0.2, callbacks=callbacks_list)

Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.2188 - loss: 3.6662 - val_accuracy: 0.6121 - val_loss: 2.8614
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.6395 - loss: 2.5403 - val_accuracy: 0.6572 - val_loss: 1.8562
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.6768 - loss: 1.6204 - val_accuracy: 0.7134 - val_loss: 1.4225
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.7522 - loss: 1.1955 - val_accuracy: 0.7379 - val_loss: 1.2251
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.7981 - loss: 0.9396 - val_accuracy: 0.7513 - val_loss: 1.1225
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - accuracy: 0.8315 - loss: 0.7956 - val_accuracy: 0.7674 - val_loss: 1.0622
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.8620 - loss: 0.6537 - val_accuracy: 0.7807 - val_loss: 1.0257
Epoch 8/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.8900 - loss: 0.5468 - val_accuracy: 0.7830 - 

In [ ]:
model3.evaluate(X_test, y_test)

71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7923 - loss: 1.0223


[1.0398054122924805, 0.790739119052887]

In [ ]:
model3 = keras.models.load_model("checkpoint_model.keras")
model3.evaluate(X_test, y_test)

71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7923 - loss: 1.0223


[1.0398054122924805, 0.790739119052887]

# Custom Callbacks
Define your own monitoring logic:

on_epoch_begin(epoch, logs)     
on_epoch_end(epoch, logs)       
on_batch_begin(batch, logs)     
on_batch_end(batch, logs)        
on_train_begin(logs)            
on_train_end(logs)

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        print("Starting training...")
        self.per_epoch_losses = []

    def on_epoch_begin(self, epoch, logs=None):
        print(f"\nStarting epoch {epoch + 1}")

    def on_epoch_end(self, epoch, logs=None):
        # Store the loss for this epoch
        self.per_epoch_losses.append(logs.get('loss'))
        print(f"\nEpoch {epoch + 1} completed")
        print(f"Loss: {logs.get('loss'):.4f}")
        print(f"Accuracy: {logs.get('accuracy'):.4f}")

    def on_train_end(self, logs=None):
        print("\nTraining completed!")
        print(f"Final loss: {self.per_epoch_losses[-1]:.4f}")

custom_callback = CustomCallback()

# Test the custom callback
model4 = build_model()
model4.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model4.fit(X_train, y_train, epochs=5, batch_size=512, validation_split=0.2, callbacks=[custom_callback], verbose=False)


Starting training...

Starting epoch 1

Epoch 1 completed
Loss: 3.3195
Accuracy: 0.4635

Starting epoch 2

Epoch 2 completed
Loss: 2.0639
Accuracy: 0.5985

Starting epoch 3

Epoch 3 completed
Loss: 1.4320
Accuracy: 0.6914

Starting epoch 4

Epoch 4 completed
Loss: 1.1170
Accuracy: 0.7713

Starting epoch 5

Epoch 5 completed
Loss: 0.9047
Accuracy: 0.8088

Training completed!
Final loss: 0.9047
